# Worksheet 08

Name:  Erwin Pimentel
UID: U97810050

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[3.4855948784135293, 5.450107731995901, 5.063307688710633, 4.166451656936867, 3.859396014970198, 4.359318965442422, 4.146960742675675, 4.846919499519045, 4.696592955067888, 5.952591679079797]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [7]:
s2 = np.random.normal(8 , 1, 10).tolist()
print(s2)

[7.999261474706113, 7.924494700228486, 7.789711857359639, 7.627636420439224, 9.117066156491397, 7.821495627416395, 8.333197541937661, 7.020049793300054, 7.1433144883291355, 7.721975757770691]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.952591679079797, 4.696592955067888, 4.846919499519045, 7.642733340034468, 4.146960742675675, 4.359318965442422, 3.859396014970198, 4.166451656936867, 5.063307688710633, 5.450107731995901]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

s1 = [3.4855948784135293, 5.450107731995901, 5.063307688710633, 4.166451656936867, 3.859396014970198, 4.359318965442422, 4.146960742675675, 4.846919499519045, 4.696592955067888, 5.952591679079797]
s1_mean = 5
s1_var = 1
s2 = [7.9012152639409825, 8.860695416461592, 6.7925650215683175, 6.562114303409873, 8.512729988109461, 8.55256138021691, 7.713169666521784, 8.0595127752344, 7.565737569969725, 7.642733340034468]
s2_mean = 8
s2_var = 1
coin_output = [0, 1]
probability of choosing from s1 or s2 = 1/2 (50%)

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [9]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


[4.696592955067888, 4.846919499519045, 4.146960742675675, 4.359318965442422, 3.859396014970198, 4.166451656936867, 5.063307688710633, 5.450107731995901]
[5.952591679079797, 7.642733340034468]
P(S_1) = 0.8,  P(S_2) = 0.2
mean_1 = 4.573631906914829,  mean_2 = 6.797662509557132
var_1 = 0.25021309443162437,  var_2 = 0.7141447085236536


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [10]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.952591679079797
probability of observing that point if it came from cluster 0 =  4.04814628294486e-07
probability of observing that point if it came from cluster 1 =  0.2773688467172279
point =  4.696592955067888
probability of observing that point if it came from cluster 0 =  1.413055518724508
probability of observing that point if it came from cluster 1 =  0.007371155231859723
point =  4.846919499519045
probability of observing that point if it came from cluster 0 =  0.8781237224288544
probability of observing that point if it came from cluster 1 =  0.013392880562648287
point =  7.642733340034468
probability of observing that point if it came from cluster 0 =  3.404305423867811e-33
probability of observing that point if it came from cluster 1 =  0.2773688467172279
point =  4.146960742675675
probability of observing that point if it came from cluster 0 =  0.37254299154777837
probability of observing that point if it came from cluster 1 =  0.0005695387599861814
point =  4.35

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [12]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [ sum(map(lambda x : (x - mean[0])**2, prob_s0_x)) / len(prob_s0_x) , sum(map(lambda x : (x - mean[1])**2, prob_s1_x)) / len(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.8,  P(S_2) = 0.2
mean_1 = 4.463735371104114,  mean_2 = 6.3701704164306
var_1 = 14.289254216219827,  var_2 = 37.148251019039236


These values are a bit different from the original ones i got from K means. For starters the variance of this dataset is much higher than the original values, however the mean has seemingly stayed around the same value, slightly lower even than the original values obtained from K means.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [14]:
# Initialize lists to store updated conditional probabilities
updated_prob_s0_x = []
updated_prob_s1_x = []

for i in range(len(data)):
    # Calculate the updated conditional probabilities using the new means and variances
    p_s0 = norm.pdf(data[i], mean[0], var[0]) * prob_c[0]
    p_s1 = norm.pdf(data[i], mean[1], var[1]) * prob_c[1]
    
    # Normalize the probabilities to ensure they sum to 1
    total_prob = p_s0 + p_s1
    p_s0 /= total_prob
    p_s1 /= total_prob
    
    # Append the updated probabilities to the lists
    updated_prob_s0_x.append(p_s0)
    updated_prob_s1_x.append(p_s1)

# Print the updated conditional probabilities for a few data points
for i in range(10):  # Print the updated probabilities
    print("Data Point:", data[i])
    print("Updated Probability of coming from S_1 =", updated_prob_s0_x[i])
    print("Updated Probability of coming from S_2 =", updated_prob_s1_x[i])
    print()

# Calculate and print the mean and variance of the updated probabilities
mean_updated_prob_s0_x = sum(updated_prob_s0_x) / len(updated_prob_s0_x)
mean_updated_prob_s1_x = sum(updated_prob_s1_x) / len(updated_prob_s1_x)
var_updated_prob_s0_x = sum((p - mean_updated_prob_s0_x) ** 2 for p in updated_prob_s0_x) / len(updated_prob_s0_x)
var_updated_prob_s1_x = sum((p - mean_updated_prob_s1_x) ** 2 for p in updated_prob_s1_x) / len(updated_prob_s1_x)

print("Mean of Updated Probability of S_1 =", mean_updated_prob_s0_x)
print("Variance of Updated Probability of S_1 =", var_updated_prob_s0_x)
print("Mean of Updated Probability of S_2 =", mean_updated_prob_s1_x)
print("Variance of Updated Probability of S_2 =", var_updated_prob_s1_x)


Data Point: 5.952591679079797
Updated Probability of coming from S_1 = 0.8630378507503271
Updated Probability of coming from S_2 = 0.1369621492496729

Data Point: 4.696592955067888
Updated Probability of coming from S_1 = 0.8637746019005288
Updated Probability of coming from S_2 = 0.13622539809947123

Data Point: 4.846919499519045
Updated Probability of coming from S_1 = 0.8637274224191355
Updated Probability of coming from S_2 = 0.13627257758086445

Data Point: 7.642733340034468
Updated Probability of coming from S_1 = 0.8608009319694558
Updated Probability of coming from S_2 = 0.13919906803054424

Data Point: 4.146960742675675
Updated Probability of coming from S_1 = 0.8638526054603104
Updated Probability of coming from S_2 = 0.13614739453968966

Data Point: 4.359318965442422
Updated Probability of coming from S_1 = 0.8638400501368995
Updated Probability of coming from S_2 = 0.1361599498631005

Data Point: 3.859396014970198
Updated Probability of coming from S_1 = 0.8638343279613278


h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [15]:
hard_assignment = []  # List to store hard cluster assignments (0 or 1)

for i in range(len(data)):
    # Compare the conditional probabilities and assign the point to the cluster with the higher probability
    if updated_prob_s0_x[i] > updated_prob_s1_x[i]:
        hard_assignment.append(0)
    else:
        hard_assignment.append(1)

# Print the hard cluster assignments for each data point
for i in range(len(data)):
    print("Data Point:", data[i], "Cluster Assignment:", hard_assignment[i])


Data Point: 5.952591679079797 Cluster Assignment: 0
Data Point: 4.696592955067888 Cluster Assignment: 0
Data Point: 4.846919499519045 Cluster Assignment: 0
Data Point: 7.642733340034468 Cluster Assignment: 0
Data Point: 4.146960742675675 Cluster Assignment: 0
Data Point: 4.359318965442422 Cluster Assignment: 0
Data Point: 3.859396014970198 Cluster Assignment: 0
Data Point: 4.166451656936867 Cluster Assignment: 0
Data Point: 5.063307688710633 Cluster Assignment: 0
Data Point: 5.450107731995901 Cluster Assignment: 0
